<a href="https://colab.research.google.com/github/evenikkal/mitp/blob/main/8_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastapi import FastAPI, HTTPException, status
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime
from uuid import uuid4

app = FastAPI(
    title="Task Manager API",
    description="Мини-микросервис для управления задачами",
    version="1.0.0"
)

class TaskCreate(BaseModel):

    title: str = Field(..., min_length=1, max_length=100, description="Название задачи")
    description: Optional[str] = Field(None, max_length=500, description="Описание задачи")
    priority: int = Field(default=1, ge=1, le=5, description="Приоритет (1-5)")


class TaskUpdate(BaseModel):

    title: Optional[str] = Field(None, min_length=1, max_length=100)
    description: Optional[str] = Field(None, max_length=500)
    priority: Optional[int] = Field(None, ge=1, le=5)
    completed: Optional[bool] = None


class Task(BaseModel):

    id: str
    title: str
    description: Optional[str]
    priority: int
    completed: bool
    created_at: datetime
    updated_at: datetime


tasks_db: dict[str, Task] = {}


@app.get("/", tags=["Health"])
def root():
    """Проверка работоспособности сервиса"""
    return {"status": "ok", "service": "Task Manager API"}


@app.get("/health", tags=["Health"])
def health_check():
    """Health check эндпоинт"""
    return {"status": "healthy", "tasks_count": len(tasks_db)}


@app.post("/tasks", response_model=Task, status_code=status.HTTP_201_CREATED, tags=["Tasks"])
def create_task(task_data: TaskCreate):
    """Создать новую задачу"""
    now = datetime.now()
    task = Task(
        id=str(uuid4()),
        title=task_data.title,
        description=task_data.description,
        priority=task_data.priority,
        completed=False,
        created_at=now,
        updated_at=now
    )
    tasks_db[task.id] = task
    return task


@app.get("/tasks", response_model=list[Task], tags=["Tasks"])
def get_all_tasks(completed: Optional[bool] = None, priority: Optional[int] = None):

    tasks = list(tasks_db.values())

    if completed is not None:
        tasks = [t for t in tasks if t.completed == completed]
    if priority is not None:
        tasks = [t for t in tasks if t.priority == priority]

    return sorted(tasks, key=lambda t: (-t.priority, t.created_at))


@app.get("/tasks/{task_id}", response_model=Task, tags=["Tasks"])
def get_task(task_id: str):
    """Получить задачу по ID"""
    if task_id not in tasks_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Задача с ID '{task_id}' не найдена"
        )
    return tasks_db[task_id]


@app.patch("/tasks/{task_id}", response_model=Task, tags=["Tasks"])
def update_task(task_id: str, task_data: TaskUpdate):
    """Частично обновить задачу"""
    if task_id not in tasks_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Задача с ID '{task_id}' не найдена"
        )

    task = tasks_db[task_id]
    update_data = task_data.model_dump(exclude_unset=True)

    updated_task = task.model_copy(update={
        **update_data,
        "updated_at": datetime.now()
    })

    tasks_db[task_id] = updated_task
    return updated_task


@app.delete("/tasks/{task_id}", status_code=status.HTTP_204_NO_CONTENT, tags=["Tasks"])
def delete_task(task_id: str):
    """Удалить задачу"""
    if task_id not in tasks_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Задача с ID '{task_id}' не найдена"
        )
    del tasks_db[task_id]


@app.post("/tasks/{task_id}/complete", response_model=Task, tags=["Tasks"])
def complete_task(task_id: str):
    """Отметить задачу как выполненную"""
    if task_id not in tasks_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Задача с ID '{task_id}' не найдена"
        )

    task = tasks_db[task_id]
    updated_task = task.model_copy(update={
        "completed": True,
        "updated_at": datetime.now()
    })
    tasks_db[task_id] = updated_task
    return updated_task

if __name__ == "__main__":
    import uvicorn
    uvicorn.run("main:app", host="0.0.0.0", port=8000, reload=True)


In [ ]:
import click

from mycli import __version__


@click.group()
@click.version_option(version=__version__, prog_name="mycli")
def main():
    pass


@main.command()
@click.argument("name")
@click.option("--count", "-c", default=1, help="Количество приветствий")
def greet(name: str, count: int):
    for _ in range(count):
        click.echo(f"Привет, {name}!")


@main.command()
@click.argument("numbers", nargs=-1, type=float, required=True)
@click.option("--precision", "-p", default=2, help="Точность округления")
def calc(numbers: tuple[float, ...], precision: int):
    total = sum(numbers)
    avg = total / len(numbers)

    click.echo(f"Числа: {', '.join(str(n) for n in numbers)}")
    click.echo(f"Сумма: {round(total, precision)}")
    click.echo(f"Среднее: {round(avg, precision)}")


@main.command()
@click.argument("input_file", type=click.Path(exists=True))
@click.option("--output", "-o", type=click.Path(), help="Файл для записи")
@click.option("--upper/--no-upper", default=False, help="Преобразовать в верхний регистр")
def process(input_file: str, output: str | None, upper: bool):
    with open(input_file, "r", encoding="utf-8") as f:
        content = f.read()

    if upper:
        content = content.upper()

    lines = len(content.splitlines())
    words = len(content.split())
    chars = len(content)

    click.echo(f"Строк: {lines}, Слов: {words}, Символов: {chars}")

    if output:
        with open(output, "w", encoding="utf-8") as f:
            f.write(content)
        click.echo(f"Результат сохранён в {output}")


if __name__ == "__main__":
    main()
